In [1]:
!pip install pycep_correios
!pip install geopy

     |████████████████████████████████| 100 kB 4.7 MB/s 
     |████████████████████████████████| 6.3 MB 41.4 MB/s 
     |████████████████████████████████| 45 kB 1.9 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
import pandas as pd
import pycep_correios
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [14]:
app_name = "meu_bndes_app"

In [15]:
# Return address from zip code
def get_address_by_zip(zip):
    address = pycep_correios.get_address_from_cep(zip)
    full_address = address['logradouro'] + ", " + address['bairro'] + ", " + address['cidade'] + " - " + address['uf']
    return full_address

# Return location by address
def get_geolocation_by_address(address):
    geolocator = Nominatim(user_agent=app_name)
    return geolocator.geocode(address)

# Return coordinates by address
def get_coordinates_by_address(address):
    return get_geolocation_by_address(address).point

# Return coordinates by zip
def get_coordinates_by_zip(zip):
    address = get_address_by_zip(zip)
    return get_geolocation_by_address(address).point

In [16]:
get_coordinates_by_zip('31170360')

Point(-19.8863658, -43.927372, 0.0)

In [18]:
# DataFrame test
df = pd.DataFrame({'CEP': ['31170360', '31080300']})

# Troque test_app pelo nome da sua aplicação/sistema
# geolocator = Nominatim(user_agent="meu_bndes_app")
# geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Cria a coluna address com os endereços retornados a partir do CEP
df['address'] = df['CEP'].apply(get_address_by_zip)

# Cria a coluna location com o local retornado a partir do endereço
df['geocode'] = df['address'].apply(get_geolocation_by_address)

# Seleciona a latitude e longitude que está dentro do local
df['loc'] = df['geocode'].apply(lambda loc: tuple(loc.point) if loc else None)
df

,CEP,address,geocode,loc
0,31170360,"Rua Júlio Pereira da Silva, Cidade Nova, Belo ...","(Rua Júlio Pereira da Silva, Cidade Nova, Regi...","(-19.8863658, -43.927372, 0.0)"
1,31080300,"Rua Matutina, Santa Inês, Belo Horizonte - MG","(Rua Matutina, Ana Lúcia, Santa Inês, Regional...","(-19.8840787, -43.9079954, 0.0)"
